<a href="https://colab.research.google.com/github/ayu1729/active_learning_trial/blob/main/ActiveLearning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
classifier = KerasClassifier(create_keras_model)

(1000, 28, 28, 1)


In [ ]:
print(y_initial.shape)

(1000, 10)


32/32 [==============================] - 3s 78ms/step - loss: 2.3158 - accuracy: 0.0982


In [ ]:
learner.fit(X_initial,y_initial)

32/32 [==============================] - 3s 79ms/step - loss: 2.3167 - accuracy: 0.1244


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


ActiveLearner(X_training=array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0....
              bootstrap_init=None,
 

In [ ]:
pip install modAL

In [ ]:
#keras.utils.np_utils.to_categorical
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from modAL.models import ActiveLearner


# build function for the Keras' scikit-learn API
def create_keras_model():
    """
    This function compiles and returns a Keras model.
    Should be passed to KerasClassifier in the Keras scikit-learn API.
    """

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

    return model


# create the classifier
classifier = KerasClassifier(create_keras_model)

"""
Data wrangling
1. Reading data from Keras
2. Assembling initial training data for ActiveLearner
3. Generating the pool
"""

# read training data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 28, 28, 1).astype('float32') / 255
X_test = X_test.reshape(10000, 28, 28, 1).astype('float32') / 255
y_train = keras.utils.np_utils.to_categorical(y_train, 10)
y_test = keras.utils.np_utils.to_categorical(y_test, 10)

# assemble initial data
n_initial = 1000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
X_initial = X_train[initial_idx]
y_initial = y_train[initial_idx]

# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)

"""
Training the ActiveLearner
"""

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=X_initial, y_training=y_initial,
    verbose=1
)

# the active learning loop
n_queries = 10
for idx in range(n_queries):
    query_idx, query_instance = learner.query(X_pool, n_instances=100, verbose=0)
    print(query_idx)
    learner.teach(
        X=X_pool[query_idx], y=y_pool[query_idx], only_new=True,
        verbose=1
    )
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)

# the final accuracy score
print(learner.score(X_test, y_test, verbose=1))

32/32 [==============================] - 4s 80ms/step - loss: 2.3029 - accuracy: 0.1121
[46205 37827 47456 51356 33878 12994 37079  3614 29337  8699 48152 33697
 32201 17315 17319 54174 19308 53994 18679 14574  7223 36763  4710 49100
 57833 13661 51756 42147 26643 53302 42851 46425 53166 33937 45171 27763
 12353 27521  2975  4127 25234 49718  4393 34854 38035 30263  8263 15040
 39676 18960  8387 30186 47243 29599 52357 22793 39095 24905 46214 21350
 22738 58506 11146   154 10615 13835 42609 15438 40186 58912 12316 52882
 53833 34903 35280 49011 15855 22482 23928 15596 31966  8181 40673 15747
 49360 54378  8572 36955 52903  9136 34578 49303 19786 56899 11122 32773
 30774  7149 28773 35154]
4/4 [==============================] - 1s 60ms/step - loss: 2.2946 - accuracy: 0.0737
[40446 31610  7145 51213 31027 31050 53312 17159  9983 46221 50798 13455
 21350 16608  8608 30050 41463 30666 30674 29135 52206 14765  7176 28547
 49493  7178 23719  3404 40590 45096 33312  7986 56052  1129  1505 576